In [1]:
import sys
sys.path.append(r"../")
sys.path.append(r"../../utils")
import pandas as pd
from psychodiagnostic_computations import compute_swan_scores, compute_sdq_scores, compute_cbcl_scores
from utils_Irit import calc_t_scores
from Variables import rename_align, Irit_columns_to_drop
import os
import numpy as np

In [2]:
df = pd.read_excel(r"data/ADHD.xlsx")
df = df.drop(Irit_columns_to_drop, axis=1)

rename_align_r = {j: i for i, j in rename_align.items()}
df = df.rename(rename_align_r, axis=1)


In [3]:
df, params_swan = compute_swan_scores(df)
df, params_cbcl = compute_cbcl_scores(df)

# df, param_t_score = calc_t_scores(df)


In [4]:
swan_params_to_remove_if_missing = [i for i in params_swan if i != 'ratio_of_missing_swan_values']
cbcl_params_to_remove_if_missing = [i for i in params_cbcl if i != 'ratio_of_missing_cbcl_values']
cbcl_params_to_remove_if_missing = cbcl_params_to_remove_if_missing # + param_t_score

In [5]:
df['suicidal_ideation'] = df['t1_p_rawscore_cbcl_91'] > 0
df['suicidal_behavior'] = df['t1_p_rawscore_cbcl_18'] > 0

df.loc[df['ratio_of_missing_swan_values'] > 0.5, swan_params_to_remove_if_missing] = np.nan
df.loc[df['ratio_of_missing_cbcl_values'] > 0.5, cbcl_params_to_remove_if_missing] = np.nan


In [6]:

directory = "data/preprocessed"
if not os.path.exists(directory):
    os.makedirs(directory)

filepath = os.path.join(directory, "ADHD.csv")
df.to_csv(filepath, index=False)